In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import argparse
import cv2
import imutils
import time
import pandas as pd
import os 

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,MaxPooling2D,Activation,Conv2D

In [12]:
data=r'C:\Users\harsh\Downloads\content\Hollywood2_final\labels\train'
list=[]
CATEGORIES=os.listdir(data)
for category in CATEGORIES:
    listi=[]
    path=os.path.join(data,category)
    df=pd.read_csv(path)
    arr=df.values
    for i in arr:
        for j in i:
            if j[len(j)-2]=='-':
                listi.append(-1)
            else:
                listi.append(1)
    list.append(listi)

list_new=[]
for i in range (0,823):
    list_temp=[]
    for j in list:
        list_temp.append(j[i])
    list_new.append(list_temp)
    
Y=[]
for i in list_new:
    listi=[]
    for j in i:
        if j==-1:
            listi.append(0)
        else:
            listi.append(1)
    Y.append(listi) 

Y=np.array(Y)

print(Y.shape)


    
    

(823, 12)


In [13]:
data_vid=r'C:\Users\harsh\Downloads\content\Hollywood2_final\videos\train'
os.listdir(data_vid)
training_data=[]
img_size=50
for vid in os.listdir(data_vid):
    vid_temp=os.path.join(data_vid,vid)
    cap = cv2.VideoCapture(vid_temp)
    
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    seconds = round(frames / fps)
    
    cap.set(cv2.CAP_PROP_POS_MSEC,int(seconds/2)*100)
    while (cap.isOpened()):
        ret,frame=cap.read()
        image=frame
        img_new=cv2.resize(image,(img_size,img_size))
        image_final = cv2.cvtColor(img_new, cv2.COLOR_BGR2GRAY)
        training_data.append(image_final)
        break

training_data=np.array(training_data).reshape(-1,img_size,img_size,1) 


In [14]:
training_data.shape

(823, 50, 50, 1)

In [ ]:
model=Sequential()


model.add(    Conv2D(64,  (3,3)  ,input_shape=training_data.shape[1:])     )
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(    Conv2D(64,  (3,3)  )     )
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(12))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(training_data,Y,batch_size=32,epochs=12)

Epoch 1/12
26/26 [==============================] - 3s 68ms/step - loss: 2.0157 - accuracy: 0.1191
Epoch 2/12
26/26 [==============================] - 1s 51ms/step - loss: 0.3394 - accuracy: 0.1713
Epoch 3/12
26/26 [==============================] - 1s 56ms/step - loss: 0.2776 - accuracy: 0.3329
Epoch 4/12
14/26 [===============>..............] - ETA: 0s - loss: 0.2296 - accuracy: 0.4665

In [ ]:
data_vid=r'C:\Users\harsh\Downloads\content\Hollywood2_final\videos\test'
os.listdir(data_vid)
testing_data=[]
img_size=50
for vid in os.listdir(data_vid):
    vid_temp=os.path.join(data_vid,vid)
    cap = cv2.VideoCapture(vid_temp)
    
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    seconds = round(frames / fps)
    
    cap.set(cv2.CAP_PROP_POS_MSEC,int(seconds/2)*100)
    while (cap.isOpened()):
        ret,frame=cap.read()
        image=frame
        img_new=cv2.resize(image,(img_size,img_size))
        image_final = cv2.cvtColor(img_new, cv2.COLOR_BGR2GRAY)
        testing_data.append(image_final)
        break

testing_data=np.array(testing_data).reshape(-1,img_size,img_size,1) 

In [ ]:
data=r'C:\Users\harsh\Downloads\content\Hollywood2_final\labels\test'
list=[]
CATEGORIES=os.listdir(data)

for category in CATEGORIES:
    listi=[]
    path=os.path.join(data,category)
    df=pd.read_csv(path)
    arr=df.values
    for i in arr:
        for j in i:
            if j[len(j)-2]=='-':
                listi.append(-1)
            else:
                listi.append(1)
    list.append(listi)
   

list_new=[]
for i in range (0,884):
    list_temp=[]
    for j in list:
        list_temp.append(j[i])
    list_new.append(list_temp)
   
    
Y_test=[]
for i in list_new:
    listi=[]
    for j in i:
        if j==-1:
            listi.append(0)
        else:
            listi.append(1)
    Y_test.append(listi) 

Y_test=np.array(Y_test)  

In [ ]:
predictions=model.predict(testing_data)

In [ ]:
pred_test=[]
for i in predictions:
    listi=[]
    for j in i:
        if j<0.5:
            listi.append(0)
        else:
            listi.append(1)
    pred_test.append(listi)
pred_test=np.array(pred_test)    

In [ ]:
count =0
for i in  range (0,884):
    add=1
    for j in pred_test[i]==Y_test[i]:
        if j==False:
            add=0
            break
    count=count+add
print(count/884)    